## DESCARGA DE DATOS DE INCIDENCIA DELICTIVA

In [2]:
# Instalando las paqueterías necesarias.
%pip install pandas openpyxl requests beautifulsoup4
%pip install wget
%pip install gdown
%pip install unidecode


[notice] A new release of pip available: 22.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Las siguientes librerías nos ayudaran para el desarrollo del proceso de descarga y generación del archivo txt. es necesario que este al principio para su correcto funcionamiento.

Se utiliza librerías especiales como gwdown para descarga de archivos grandes, eliminando la confirmación de permiso de descarga.

Se utiliza librería unicode, para poder quitar los acentos de ciertas palabras y usarlo como nombre de archivos a descargar.


In [3]:
# Importación de librerías para usarlas en el proyecto de descarga y generación del txt con la descripción de esta
import pandas as pd
import wget
import gdown
import requests
import re
import os
import time
from bs4 import BeautifulSoup
from datetime import datetime
from logging import exception
from unidecode import unidecode
from datetime import datetime, timedelta

La inicialización de ciertas variables ayudaran a preparar la información inicial de cada una de ellas, además de que puedan ser usadas en cualquier parte del código o proceso, sin en el que perjudique el ámbito o espacio donde estas estén siendo seteadas.

In [4]:
# Inicializando variables que serán usadas más adelante

incidencia_delictiva_link = 'https://www.gob.mx/sesnsp/acciones-y-programas/datos-abiertos-de-incidencia-delictiva'
incidencia_title = ''
incidencia_description = ''

folder_path_data = '../rawdata'
folder_path_log = '../logs/descarga/'
folder_path_error_log = '../logs/errores/'
csv_filepath = ''

# Creando variable que contendra el listado de cada source descargado.
sources_list = []

# Creando variable que contendra el listado de cada nombre de archivo descargado y que esta siendo enviado a nuestro repositorio
names_files_list = []

texto_list = ''

#### Sección donde se encuentran las funciones generales del proceso

In [5]:
def log_error_write(description, line):
    # Verificando si la carpeta existe, si no, crearla
    if not os.path.exists(folder_path_error_log):
        os.makedirs(folder_path_error_log)
        
    date_now = datetime.now()

    # Variable que contiene el texto que tendrá el archivo txt que se generará y descargará.
    line_error = f"""{date_now.strftime('%Y-%m-%d %H:%M:%S')} {line} - {description}"""
    # Se crea el archivo a partir del texto de arriba, el cual se guardará con el nombre descriptivo más la fecha día*mes*año, para identificar la descarga por día.
    with open(f'{folder_path_error_log}/ddf_err{date_now.strftime("%d%m%y")}.txt', 'a', encoding="utf-8") as error_file:
        error_file.write(f"\n{line_error}")

def log_write(description, line):
    # Verificando si la carpeta existe, si no, crearla
    if not os.path.exists(folder_path_log):
        os.makedirs(folder_path_log)
        
    date_now = datetime.now()

    # Variable que contiene el texto que tendrá el archivo txt que se generará y descargará.
    line_write = f"""{date_now.strftime('%Y-%m-%d %H:%M:%S')} {line} - {description}"""
    # Se crea el archivo a partir del texto de arriba, el cual se guardará con el nombre descriptivo más la fecha día*mes*año, para identificar la descarga por día.
    with open(f'{folder_path_log}/muncoord{date_now.strftime("%d%m%y")}.txt', 'a', encoding="utf-8") as write_file:
        write_file.write(f"\n{line_write}")

def create_csv_from_DataFrame(df, csv_filename):
    try:

        # Construyendo la ruta completa del archivo
        csv_filepath = os.path.join(folder_path_data, csv_filename)

        # Guardando el DataFrame como un archivo CSV
        df.to_csv(csv_filepath, index=False, encoding='utf-8')
        print(f"DataFrame guardado exitosamente en {csv_filepath}")

    except Exception as e:
        # Imprimiendo un mensaje de error en caso de excepción
        log_error_write(f"Error al guardar el DataFrame como CSV: {e}", 'create_csv_from_DataFrame')
        print(f"Error al guardar el DataFrame como CSV: {e}")

#### Sección donde se encuentran las funciones para los distintos metodos de descarga

In [6]:
# sección de funciones para descarga de archivos desde fuentes que vienen de enlaces proporcionados por la página del gobierno.

# Función utilizada para la descarga de archivos almacenados en google drive por parte de la institución donde se requiere obtener los datos.
# - document: recibe el id del documento en google drive.
# - description: Texto de la etiqueta de enlace, el cual describe la naturaleza de la información a descargar.
def download_csv_from_http(document, description):

  print(f"link " + description)

  try:
    # Verificando si la carpeta existe, si no, crearla
    if not os.path.exists(folder_path_data):
        os.makedirs(folder_path_data)

    # Url de descarga a traves de google drive
    csv_url = f'https://drive.google.com/uc?id={document}'

    # Descargando archivos desde la url proporcionada
    response_csv = requests.get(csv_url)

    # Parte donde se hace una limpieza del parámetro description enviado,
    # esto para construir el nombre que tendrá el archivo csv que se descarga a partir de los datos de las fuentes
    # Utilizando expresión regular para reemplazar caracteres especiales y espacios por un solo guion bajo aplicándolo en la descripción del enlace.
    csv_filename = re.sub(r'[^\w\s]+', '', description)
    csv_filename = re.sub(r'\s+', '', csv_filename)
    # Utilizando unidecode para quitar los acentos y no cause conflictos como nombre de archivo
    csv_filename = unidecode(csv_filename)
    csv_filename = csv_filename + ".csv"

    # Construyendo la ruta completa del archivo
    csv_filepath = os.path.join(folder_path_data, csv_filename)

    names_files_list.append(csv_filename)
    #------------------------------------------------------------------------------------------------------------------------------------

    # Guardando el archivo descargado, en el repositorio del proyecto
    with open(csv_filepath, 'wb') as csv_file:
      csv_file.write(response_csv.content)

    # Leer el archivo CSV con la codificación original
    df = pd.read_csv(csv_filepath, encoding='latin1')

    # Guardar el archivo CSV con codificación UTF-8
    df.to_csv(csv_filepath, index=False, encoding='utf-8')
  
  except ValueError as ve:
    # Manejo de errores para entradas no válidas (por ejemplo, si no se ingresa un número)
    log_error_write(f"Error de valor: {ve}", 'download_csv_from_http')
    print(f"Error de valor: {ve}")

  except Exception as e:
    # Manejo de errores para cualquier otro tipo de excepción no anticipada
    log_error_write(f"Ocurrió un error inesperado: {e}", 'download_csv_from_http')
    print(f"Ocurrió un error inesperado: {e}")

  else:
    # Este bloque se ejecuta si no se produce ninguna excepción en el bloque try
    print("¡Proceso download_csv_from_http de descarga completado!")

  finally:
    # Este bloque siempre se ejecuta, independientemente de si se produjo una excepción o no
    print("¡Fin de la descarga!")

# *****************************************************************************************************************************************

# Función utilizada para la descarga de archivos almacenados en google drive por parte de la institución donde se requiere obtener los datos.
# - document: recibe el id del documento en google drive.
# - description: Texto de la etiqueta de enlace, el cual describe la naturaleza de la información a descargar.
# (Para descarga de documentos muy grandes, libreria gdown)
def download_csv_from_http_high(document, description):

  print(f"link " + description)

  try:
    # Verificando si la carpeta existe, si no, crearla
    if not os.path.exists(folder_path_data):
      os.makedirs(folder_path_data)

    # Url de descarga a traves de google drive
    csv_url = f'https://drive.google.com/uc?id={document}'

    # Parte donde se hace una limpieza del parámetro description enviado,
    # esto para construir el nombre que tendrá el archivo csv que se descarga a partir de los datos de las fuentes
    # Utilizando expresión regular para reemplazar caracteres especiales y espacios por un solo guion bajo aplicándolo en la descripción del enlace.
    csv_filename = re.sub(r'[^\w\s]+', '', description)
    csv_filename = re.sub(r'\s+', '', csv_filename)
    csv_filename = unidecode(csv_filename)
    csv_filename = csv_filename + ".csv"

    # Construyendo la ruta completa del archivo
    csv_filepath = os.path.join(folder_path_data, csv_filename)

    names_files_list.append(csv_filename)
    #----------------------------------------------------------------------------------------------------------------------------------------

    # Utilizando wget para descargar archivos grandes sin restricciones
    gdown.download(csv_url, csv_filepath, quiet=False)

    # Leer el archivo CSV con la codificación original
    df = pd.read_csv(csv_filepath, encoding='latin1')

    # Guardar el archivo CSV con codificación UTF-8
    df.to_csv(csv_filepath, index=False, encoding='utf-8')
  
  except ValueError as ve:
    # Manejo de errores para entradas no válidas (por ejemplo, si no se ingresa un número)
    log_error_write(f"Error de valor: {ve}", 'download_csv_from_http_high')
    print(f"Error de valor: {ve}")

  except Exception as e:
    # Manejo de errores para cualquier otro tipo de excepción no anticipada
    log_error_write(f"Ocurrió un error inesperado: {e}", 'download_csv_from_http_high')
    print(f"Ocurrió un error inesperado: {e}")

  else:
    # Este bloque se ejecuta si no se produce ninguna excepción en el bloque try
    print("¡Proceso download_csv_from_http_high de descarga completado!")

  finally:
    # Este bloque siempre se ejecuta, independientemente de si se produjo una excepción o no
    print("¡Fin de la descarga!")

# *******************************************************************************************************************************************************************


def download_from_api_high(api_url, parameters):
    try:
        # Realizando la solicitud GET a la API con los parámetros
        response = requests.get(api_url, params=parameters)

        # Verificando si la solicitud fue exitosa (código 200)
        if response.status_code == 200:
            # Convertir la respuesta JSON en un DataFrame de pandas
            df = pd.DataFrame(response.json())

            return df

        else:
            # Imprimir un mensaje de error si la solicitud no fue exitosa
            log_error_write(f"Error al realizar la solicitud. Código de estado: {response.status_code}", 'download_from_api')
            print(f"Error al realizar la solicitud. Código de estado: {response.status_code}")

    except Exception as e:
        # Imprimir un mensaje de error en caso de excepción
        log_error_write(f"Ocurrió un error inesperado: {e}", 'download_from_api')
        print(f"Error: {e}")

    return None


#### Web Scraping para obtener las regiones geograficas en las que esta divido el estado de Sonora.

In [7]:
try:
  
  url = f'https://contraloria.sonora.gob.mx/component/zoo/category/regiones.html'
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  pretty_soup = soup.prettify()

  # Inicializando listas para las columnas del DataFrame
  municipios = []
  regiones = []

  # Se encuentra todas las etiquetas h2 que contienen nombres de regiones
  for h2_region in soup.find_all('h2', {'class': 'title'}):
      # Se extrae el nombre de la región
      nombre_region = h2_region.find('a').text.strip()

      # Se encuentra la siguiente etiqueta ul (lista) después del h2
      ul_lista = h2_region.find_next('ul', {'class': 'sub-categories'})
      
      # Iterando sobre cada li (elemento de la lista) dentro de la ul
      for li_ciudad in ul_lista.find_all('li'):
          # Extrayendo el nombre del municipio
          nombre_municipio = li_ciudad.find('a').text.strip()
          
          # Agregando el nombre del municipio y de la región a las listas
          municipios.append(nombre_municipio)
          regiones.append(nombre_region)

  # Creando el DataFrame
  regiones_df = pd.DataFrame({'Municipio': municipios, 'Region': regiones})

  # Muestra el DataFrame resultante
  print(regiones_df)

  create_csv_from_DataFrame(regiones_df, f"municipios_regiones.csv")

except ValueError as ve:
  # Manejo de errores para entradas no válidas (por ejemplo, si no se ingresa un número)
  log_error_write(f"Error de valor: {ve}", 'web scraping regiones')
  print(f"Error de valor: {ve}")

except Exception as e:
  # Manejo de errores para cualquier otro tipo de excepción no anticipada
  log_error_write(f"Ocurrió un error inesperado: {e}", 'web scraping regiones')
  print(f"Ocurrió un error inesperado: {e}")

                Municipio         Region
0                   Altar       Noraeste
1                    Átil       Noraeste
2                 Caborca       Noraeste
3                 Oquitoa       Noraeste
4               Pitiquito       Noraeste
..                    ...            ...
67             San Javier  Sierra Centro
68  San Pedro de la Cueva  Sierra Centro
69                 Soyopa  Sierra Centro
70          Suaqui Grande  Sierra Centro
71        Villa Pesqueira  Sierra Centro

[72 rows x 2 columns]
DataFrame guardado exitosamente en ../rawdata\municipios_regiones.csv


#### Web Scraping para obtener las coordenadas geográficas de todos los municipios de Sonora

In [8]:
# Inicializando listas para almacenar los datos
ciudades = []
latitudes = []
longitudes = []

try:
  
  for num in range(2):
    url = f'https://www.coordenadas.com.es/mexico/pueblos-de-sonora/26/{num + 1}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    pretty_soup = soup.prettify()

  # Encontrando el elemento tabla desde el html obtenido
    tabla = soup.find('table', class_='table')

    # Iterarando sobre las filas de la tabla
    for fila in tabla.find_all('tr'):
        # buscando y encontrando las celdas de la fila
        celdas = fila.find_all('td')
        
        # Verificando si hay celdas (evitando encabezados de la tabla)
        if celdas:
            # Extrayendo el texto de la etiqueta <a> dentro de la primera celda
            ciudad = celdas[0].find('a').text.strip()
            # Extrayendo el texto de la segunda celda
            coordenada = celdas[1].text.strip()
            
            # Dividiendo las coordenadas en latitude y longitude
            latitud, longitud = map(float, coordenada.split(','))
            
            ciudades.append(ciudad)
            latitudes.append(latitud)
            longitudes.append(longitud)

  coordenadas_df = pd.DataFrame({
     'municipio': ciudades,
     'latitude': latitudes,
     'longitude': longitudes
     })
  

  #llenando la informacion faltante

  Mmunicipios_left_df = pd.DataFrame({
     'municipio': ["Magdalena", "Mazatán", "Moctezuma", "Rayón", "Rosario", "Santa Ana", "Villa Hidalgo", "Benito Juárez"],
     'latitude': [30.733333, 28.933333, 29.806111, 29.710556, 28.05, 30.543889, 30.25, 27.1],
     'longitude': [-111.05, -110.166667, -109.679444, -110.570278, -109.3, -111.121111, -109.333333,  -109.85]
  })

  coordenadas_df = pd.concat([coordenadas_df,Mmunicipios_left_df], ignore_index = True)

  create_csv_from_DataFrame(coordenadas_df, f'municipios_coordenadas.csv')

  # Mostrar el DataFrame resultante
  print(coordenadas_df)

except ValueError as ve:
  # Manejo de errores para entradas no válidas (por ejemplo, si no se ingresa un número)
  log_error_write(f"Error de valor: {ve}", 'web scraping coordenadas')
  print(f"Error de valor: {ve}")

except Exception as e:
  # Manejo de errores para cualquier otro tipo de excepción no anticipada
  log_error_write(f"Ocurrió un error inesperado: {e}", 'web scraping coordenadas')
  print(f"Ocurrió un error inesperado: {e}")

DataFrame guardado exitosamente en ../rawdata\municipios_coordenadas.csv
        municipio   latitude   longitude
0         Aconchi  29.825040 -110.227170
1     Agua-Prieta  30.948490 -109.274370
2          Alamos  27.026940 -108.936590
3           Altar  30.713610 -111.835280
4        Arivechi  28.927990 -109.188300
..            ...        ...         ...
67          Rayón  29.710556 -110.570278
68        Rosario  28.050000 -109.300000
69      Santa Ana  30.543889 -111.121111
70  Villa Hidalgo  30.250000 -109.333333
71  Benito Juárez  27.100000 -109.850000

[72 rows x 3 columns]


#### Web Scraping , proceso de descarga de los datos de incidencia delicitva

In [9]:
# Descargando archivos proporcionados por https://www.gob.mx/sesnsp/acciones-y-programas/datos-abiertos-de-incidencia-delictiva
print('# Descargando archivos proporcionados por https://www.gob.mx/sesnsp/acciones-y-programas/datos-abiertos-de-incidencia-delictiva')
# Realizando Web scraping a la página donde están las distintas fuentes de datos
# Esto asegura que los datos puedan ser descargados de forma automática sin necesidad de estar descargado uno por uno directamente desde la página.
url = 'https://www.gob.mx/sesnsp/acciones-y-programas/datos-abiertos-de-incidencia-delictiva'

try:
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  pretty_soup = soup.prettify()

  # Obteniendo el texto dentro de la etiqueta <title>
  incidencia_source_title = soup.title.text.strip()

  # Separando el texto por los pipes "|", esto porque el texto original viene con dicho carácter especial
  title_parts = incidencia_source_title.split('|')

  # Limpiando los espacios en blanco alrededor de cada parte del titulo encontrado
  title_parts = [part.strip() for part in title_parts]

  incidencia_title = title_parts[0] + " - " + title_parts[1]

  # Buscando todas las etiquetas <h2> y extraer su texto, esto ayudara a poder extraer la descripción de la página, la explicación de la razón de la página.
  h2_tags = soup.find_all('h2')

  # Se recorren cada una de los parrafos H2 encontrados para luego extraer el texto en donde comience con "En esta pagina...".
  for h2_tag in h2_tags:
      text_inside_h2 = h2_tag.text.strip()

      if(text_inside_h2.find('En esta página')!=-1) :
        incidencia_description = text_inside_h2

  # Buscando todas las etiquetas <ul>, esto ayudara más adelante a poder realizar las descargas, pues aquí se encontrará las url de cada enlace.
  ul_tags = soup.find_all('ul')

  for ul_tag in ul_tags:
      # Buscando todas las etiquetas <li> dentro de cada <ul>
      li_tags = ul_tag.find_all('li')

      for li_tag in li_tags:
          # Buscando todas las etiquetas <a> dentro de cada <li> con texto que comienza con "Cifras"
          a_tags = li_tag.find_all('a', href=True, string=lambda x: x and x.startswith('Cifras'))

          for a_tag in a_tags:
              # Obteniendo el valor del atributo href, para extraer su url
              href_value = a_tag['href']

              # Utilizando expresión regular para extraer la parte antes de "/view" de cada url esto para conseguir el id del documento
              match = re.search(r'/file/d/(.*?)/view', href_value)
              # match = re.search(r'(.*?)/view', href_value)

              # En caso de encontrar coicidencia, se procede a extraer el id del documento de descarga.
              if match:
                extracted_part = match.group(1)
                if a_tag.text.find(str(datetime.now().year)) != -1:
                  # Condicionante para poder detectar si la descarga es de datos referentes a municipios, esto por conocimiento, son descargas más pesadas y por lo cual se ocupa de librería especial.
                  if a_tag.text.find('Municipal') != -1:
                    download_csv_from_http_high(extracted_part, a_tag.text)

                    # Imprimiendo el tag descriptivo y la url para mostrarla en pantalla
                    print(f""+ a_tag.text + ": " + href_value)
                  else:
                    download_csv_from_http(extracted_part, a_tag.text)

                  #Creando listado que será usado para el documento con la descripción de la descarga de las distintas fuentes, de incidencia delictiva
                  sources_list.append(f"- {a_tag.text}: {href_value}")
except ValueError as ve:
  # Manejo de errores para entradas no válidas (por ejemplo, si no se ingresa un número)
  log_error_write(f"Error de valor: {ve}", 'web scraping')
  print(f"Error de valor: {ve}")

except Exception as e:
  # Manejo de errores para cualquier otro tipo de excepción no anticipada
  log_error_write(f"Ocurrió un error inesperado: {e}", 'web scraping')
  print(f"Ocurrió un error inesperado: {e}")

# Descargando archivos proporcionados por https://www.gob.mx/sesnsp/acciones-y-programas/datos-abiertos-de-incidencia-delictiva
link Cifras de Incidencia Delictiva Estatal, 2015 - agosto  2023
¡Proceso download_csv_from_http de descarga completado!
¡Fin de la descarga!
link Cifras de Incidencia Delictiva Municipal, 2015 - agosto 2023. 


Downloading...
From (uriginal): https://drive.google.com/uc?id=13TjyJ9RkR49o0eWTFvhNqazeL4maORYp
From (redirected): https://drive.google.com/uc?id=13TjyJ9RkR49o0eWTFvhNqazeL4maORYp&confirm=t&uuid=fbf9c594-90df-49ec-bfc1-b4cdb3b94cfa
To: c:\UNISON\Maestria\Ingenieria_de_caracteristicas\IC-Proyecto-Indicadores-Delictivos\rawdata\CifrasdeIncidenciaDelictivaMunicipal2015agosto2023.csv
100%|██████████| 306M/306M [00:31<00:00, 9.61MB/s] 


¡Proceso download_csv_from_http_high de descarga completado!
¡Fin de la descarga!
Cifras de Incidencia Delictiva Municipal, 2015 - agosto 2023. : https://drive.google.com/file/d/13TjyJ9RkR49o0eWTFvhNqazeL4maORYp/view?usp=sharing
link Cifras de Víctimas del Fuero Común, 2015 - agosto 2023
¡Proceso download_csv_from_http de descarga completado!
¡Fin de la descarga!
link Cifras de Incidencia Delictiva Federal, 2012 - agosto 2023
¡Proceso download_csv_from_http de descarga completado!
¡Fin de la descarga!


#### Sección donde se crea el dataset que contiene los nombres de los diferentes archivos descargados

In [10]:
try:
    
    # Convertiendo la lista de nombres de archivos generados a DataFrame
    df_names = pd.DataFrame(names_files_list)

    df_names = df_names.rename(columns={0: "nombre"})

    # Guardando el DataFrame como un archivo CSV
    df_names.to_csv(f'{folder_path_data}/nombres_dataset.csv', index=False, encoding='utf-8')

except ValueError as ve:
  # Manejo de errores para entradas no válidas (por ejemplo, si no se ingresa un número)
  log_error_write(f"Error de valor: {ve}", 'Creación de dataset de nombres')
  print(f"Error de valor: {ve}")

except Exception as e:
  # Manejo de errores para cualquier otro tipo de excepción no anticipada
  log_error_write(f"Ocurrió un error inesperado: {e}", 'Creación de dataset de nombres')
  print(f"Ocurrió un error inesperado: {e}")

#### Sección para dar validación de la cantidad de archivos descargados referente a incidencia delectiva, el numero esperado a la fecha es de 4

In [11]:
try:

  nombres_dataset = pd.read_csv(f'{folder_path_data}/nombres_dataset.csv')
  df_nombres_dataset = pd.DataFrame(nombres_dataset)

  if df_nombres_dataset.value_counts().sum() != 4:
    raise Exception('Hubo cambios en la cantidad de dataset considerados')

  print("Todo en orden, en la generación de los distintos archivos")

except ValueError as ve:
  # Manejo de errores para entradas no válidas (por ejemplo, si no se ingresa un número)
  log_error_write(f"Error de valor: {ve}", 'Verificación archivos')
  print(f"Error de valor: {ve}")

except Exception as e:
  # Manejo de errores para cualquier otro tipo de excepción no anticipada
  log_error_write(f"Ocurrió un error inesperado: {e}", 'Verificación archivos')
  print(f"Ocurrió un error inesperado: {e}")

Todo en orden, en la generación de los distintos archivos


#### Sección que crea el archivo txt de la descripción de las fuentes descargadas

In [12]:
# Creando el archivo de texto con descripción general de las fuentes de descargar y parte del proceso ETL

try:
   
  # Verificando si la carpeta existe, si no, crearla
  if not os.path.exists(folder_path_log):
    os.makedirs(folder_path_log)

  # Uniendo elementos de la lista de las fuentes en donde se han descargado los datos (sources_list)
  texto_list = "\n".join(sources_list)

  date_now = datetime.now()

  # Variable que contiene el texto que tendrá el archivo txt que se generará y descargará.
  description = f"""
  Proyecto: 
  Narrando una historia con datos: explorando el poder de la ciencia de datos 

  Análisis de la variabilidad del comportamiento delictivo
  en relación con factores temporales y geográficos, y su correlación con el desarrollo social y urbano para la prevención del delito

  Equipo participante:
  - Miguel Ernesto Medina León
  - Luis Andrés Burruel Durán
  - Mario Estrada Ferreira

  Descripción de los datos descargados:
  - Las fuentes fueron descargadas y donde se puede encontrar información adicional en el sitio web: {incidencia_delictiva_link}

  Fuentes:
  {texto_list}

  Diccionario de datos:

    * Cifras de incidencia delictiva estatal, 2015 - agosto 2023

      - Año: año de registro de las averiguaciones previas y/o carpetas de investigación.
      - Clave_Ent: clave de la entidad, según el Marco Geoestadístico Nacional (MGN) del Instituto Nacional de Geografía y Estadística (INEGI).
      - Entidad: entidad federativa de registro de las averiguaciones previas y/o carpetas de investigación.
      - Bien jurídico afectado: primera clasificación de los delitos en las averiguaciones previas y/o carpetas de investigación.
      - Tipo de delito: segunda clasificación de los delitos.
      - Subtipo de delito: tercera clasificación de los delitos.
      - Modalidad: cuarta clasificación de los delitos.
      - Enero - diciembre: mes de registro de las averiguaciones previas y/o carpetas de investigación.

    * Cifras de incidencia delictiva municipal, 2015 - agosto 2023

      - Año: año de registro de las averiguaciones previas y/o carpetas de investigación.
      - Clave_Ent: clave de la entidad, según el Marco Geoestadístico Nacional (MGN) del Instituto Nacional de Geografía y Estadística (INEGI).
      - Entidad: entidad federativa de registro de las averiguaciones previas y/o carpetas de investigación.
      - Cve. Municipio: clave del municipio, según el Marco Geoestadístico Nacional (MGN) del Instituto Nacional de Geografía y Estadística (INEGI).
      - Municipio: municipio de registro de las averiguaciones previas y/o carpetas de investigación.
      - Bien jurídico afectado: primera clasificación de los delitos en las averiguaciones previas y/o carpetas de investigación. 
      - Tipo de delito: segunda clasificación de los delitos.
      - Subtipo de delito: tercera clasificación de los delitos.
      - Modalidad: cuarta clasificación de los delitos.
      - Enero - diciembre: mes de registro de las averiguaciones previas y/o carpetas de investigación.

    * Cifras de víctimas del fuero común, 2015 - agosto 2023

      - Año: año de registro de las averiguaciones previas y/o carpetas de investigación.
      - Clave_Ent: clave de la entidad, según el Marco Geoestadístico Nacional (MGN) del Instituto Nacional de Geografía y Estadística (INEGI).
      - Entidad: cntidad federativa de registro de las averiguaciones previas y/o carpetas de investigación.
      - Bien jurídico afectado: primera clasificación de los delitos en las averiguaciones previas y/o carpetas de investigación. 
      - Tipo de delito: segunda clasificación de los delitos.
      - Subtipo de delito: tercera clasificación de los delitos.
      - Modalidad: cuarta clasificación de los delitos.
      - Sexo: Sexo de las víctimas: se divide en Hombre, Mujer y No identificado.
      - Rango de edad: rangos de edades de las víctimas, clasificadas en: Adultos (18 y más), Menores de edad (0-17), No especificado y No identificado.
      - Enero - diciembre: mes de registro de las averiguaciones previas y/o carpetas de investigación.


    * Cifras de incidencia delictiva federal, 2012 - agosto 2023

      - ANO: Año de registro de las Averiguaciones Previas y/o carpetas de investigación.
      - INEGI: clave INEGI
      - ENTIDAD: Entidad federativa de registro de las Averiguaciones Previas y/o carpetas de investigación.
      - LEY: Primera clasificación de los delitos en las Averiguaciones Previas y/o carpetas de investigación. Está dividido según la ley que los regula:
      Código Penal Federal u Otras Leyes o Códigos.
      - CONCEPTO: Segunda clasificación de los delitos: para los delitos regulados por el Código Penal Federal se divide en "Contra la salud" y "Otros delitos",
      para Otras Leyes y Códigos está dividida en: "Ley General de Salud (L.G.S.)", "Ley Federal Contra la Delincuencia Organizada (L.F.C.D.O.)" y "Otras Leyes y Códigos".
      - TIPO: Tercera clasificación de los delitos: "Contra la salud" se divide en  producción, transporte, tráfico, comercio, suministro, posesión y otros; 
      "Otros delitos" se divide en Cometidos por servidores públicos, Contra el ambiente y la gestión ambiental, Contra la integridad corporal, Electorales, En materia de 
      derechos de autor, Falsedad, Titulo décimo tercero, Patrimoniales, Vías de comunicación y correspondencia y Otros delitos del C.P.F. para los delitos regulados por 
      Otras Leyes o Códigos la división es la siguiente:"Ley General de Salud (L.G.S.)" en Contra la salud en su modalidad de narcomenudeo y Otros delitos previstos en la L.G.S;
      "Ley Federal Contra la Delincuencia Organizada (L.F.C.D.O.)" en Contra la salud y Otros delitos previstos en la L.F.C.D.O y "Otras Leyes y Códigos" en Código Fiscal De La 
      Federación (C.F.F.), Ley De La Propiedad Industrial (L.P.I.), Ley De Vías Generales De Comunicación (L.V.G.C.), Ley Federal Del Derecho De Autor (L.F.D.A.), Ley Federal De 
      Armas De Fuego Y Explosivos (L.F.A.F.E.), Ley De Migración, Leyes De Instituciones De Crédito, Inversión, Fianzas Y Seguros y Otras Leyes Especiales.


  Regiones geográficas del estado de Sonora:

  - Noraeste
  - Sur
  - Sierra Alta
  - Sierra centro
  - Centro norte
  - Río Sonora

  Estas regiones geográficas han sido obtenidas de la fuente oficial: https://contraloria.sonora.gob.mx/component/zoo/category/regiones.html

  Coordenadas geográficas de los municipios de Sonora

  Estas coordenadas geográficas representan la ubicación geoespacial de los municipios dentro del estado de Sonora y han sido recuperadas de la fuente: https://www.coordenadas.com.es/mexico/pueblos-de-sonora/26/

  Fechas de descarga:
  - El proceso de descarga de los datos se realizó el: {date_now.strftime('%Y-%m-%d %H:%M:%S')}
  """
  # Se crea el archivo a partir del texto de arriba, el cual se guardará con el nombre descriptivo más la fecha día*mes*año, para identificar la descarga por día.
  with open(f'{folder_path_log}/fuentes_descripcion{date_now.strftime("%d%m%y")}_Incidencia_Delictiva.txt', 'w') as description_file:
      description_file.write(description)

  print("Archivos de datos descargados y descripción en txt creada.")

except ValueError as ve:
  # Manejo de errores para entradas no válidas (por ejemplo, si no se ingresa un número)
  log_error_write(f"Error de valor: {ve}", 'Creación de archivo de descripción general')
  print(f"Error de valor: {ve}")

except Exception as e:
  # Manejo de errores para cualquier otro tipo de excepción no anticipada
  log_error_write(f"Ocurrió un error inesperado: {e}", 'Creación de archivo de descripción general')
  print(f"Ocurrió un error inesperado: {e}")

Archivos de datos descargados y descripción en txt creada.
